In [1]:
import pandas as pd
import numpy as np
import torch
import yaml
from torch.utils.data import Dataset
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import csv
from tqdm import tqdm
import datetime
import shutil
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import random

# Set GPU memory allocation to 80%
torch.cuda.set_per_process_memory_fraction(0.8)

# Set all random seeds
def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Use before training
set_all_seeds()



In [2]:
from IPython.display import clear_output, display

def verify_and_filter_data(annotations_df, image_dir):
    """
    Verify image existence and filter annotations accordingly
    Returns filtered annotations and list of available images
    """
    available_images = set(f.split('.')[0] for f in os.listdir(image_dir) if f.endswith('.jpg'))
    filtered_df = annotations_df[annotations_df['ImageID'].isin(available_images)]
    
    print(f"Original annotations: {len(annotations_df)}")
    print(f"Available images: {len(available_images)}")
    print(f"Filtered annotations: {len(filtered_df)}")
    print(f"Unique images after filtering: {len(filtered_df['ImageID'].unique())}")
    
    return filtered_df, list(available_images)

def custom_callback(trainer):
    """Custom callback to clear output and show only current epoch"""
    clear_output(wait=True)  # Clear previous output
    print(f"\nEpoch {trainer.epoch}/{trainer.epochs}")
    print(f"{'Class':>10} {'Images':>10} {'Instances':>10} {'Box(P)':>10} {'R':>10} {'mAP50':>10} {'mAP50-95':>10}")
    print(f"{'all':>10} {len(trainer.datasets['val']):>10} {trainer.metrics['instances']:>10} "
          f"{trainer.metrics.get('metrics/precision(B)', 0):>10.4f} "
          f"{trainer.metrics.get('metrics/recall(B)', 0):>10.4f} "
          f"{trainer.metrics.get('metrics/mAP50(B)', 0):>10.4f} "
          f"{trainer.metrics.get('metrics/mAP50-95(B)', 0):>10.4f}")
    return False  # Continue training

# CELL 2: Dataset and Training Classes
class DatasetYOLO(Dataset):
    def __init__(self, annotations, image_dir, transforms=None):
        self.annotations = annotations
        self.image_dir = image_dir
        self.transforms = transforms
        self.image_ids = self.annotations['ImageID'].unique()

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_path = os.path.join(self.image_dir, f"{image_id}.jpg")
        image = Image.open(img_path).convert("RGB")
        
        # Get bounding boxes for this image
        image_annotations = self.annotations[self.annotations['ImageID'] == image_id]
        
        # Convert bounding boxes to YOLO format
        labels = []
        for _, row in image_annotations.iterrows():
            x_min, x_max = row['XMin'], row['XMax']
            y_min, y_max = row['YMin'], row['YMax']
            
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2
            width = x_max - x_min
            height = y_max - y_min
            
            labels.append([0, x_center, y_center, width, height])
            
        return img_path, torch.tensor(labels)

def save_metrics(results, results_path):
    """
    Save training metrics to CSV file
    
    Args:
        results: List of dictionaries containing metrics
        results_path: Path to save the CSV file
    """
    csv_columns = [
        'BoundingBoxCount', 
        'ValidationBoxCount',
        'TrainImageCount', 
        'ValImageCount',
        'BatchSize', 
        'ModelDirectory',
        'Precision', 
        'Recall', 
        'mAP50', 
        'mAP50-95', 
        'FinalEpoch'
    ]
    
    try:
        with open(results_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=csv_columns)
            writer.writeheader()
            for data in results:
                writer.writerow(data)
        print(f"Results saved to {results_path}")
    except Exception as e:
        print(f"Error saving metrics: {str(e)}")

def get_background_images(train_image_count, train_annotations, background_dir, random_state=42):
    """
    Get random background images that don't contain the target class
    Args:
        train_image_count: Number of training images (not boxes)
        train_annotations: Full annotations DataFrame
        background_dir: Directory containing background images
        random_state: Random seed
    """
    # Calculate number of background images needed
    num_background = int(train_image_count * (BACKGROUND_PERCENTAGE / 100))
    
    # Get all image IDs that contain any annotations
    annotated_images = set(train_annotations['ImageID'].unique())
    
    # Get all available background images
    background_files = [f[:-4] for f in os.listdir(background_dir) if f.endswith('.jpg')]
    
    # Filter out images that have annotations
    valid_backgrounds = list(set(background_files) - annotated_images)
    
    if len(valid_backgrounds) < num_background:
        print(f"Warning: Only {len(valid_backgrounds)} valid background images available")
        num_background = len(valid_backgrounds)
    
    # Randomly select background images
    random.seed(random_state)
    selected_backgrounds = random.sample(valid_backgrounds, num_background)
    
    print(f"Selected {len(selected_backgrounds)} background images")
    return selected_backgrounds

def create_yolo_dataset(annotations_subset, image_dir, output_dir, background_images=None, background_dir=None):
    """Create YOLO dataset structure and label files"""
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    
    # Group annotations by ImageID to handle multiple boxes per image
    grouped_annotations = annotations_subset.groupby('ImageID')
    
    # Process annotated images
    for image_id, image_annots in tqdm(grouped_annotations, desc="Creating dataset - annotated"):
        # Create label file with all bounding boxes for this image
        label_path = os.path.join(output_dir, 'labels', f"{image_id}.txt")
        
        with open(label_path, 'w') as f:
            for _, row in image_annots.iterrows():
                x_center = (row['XMin'] + row['XMax']) / 2
                y_center = (row['YMin'] + row['YMax']) / 2
                width = row['XMax'] - row['XMin']
                height = row['YMax'] - row['YMin']
                
                f.write(f"0 {x_center} {y_center} {width} {height}\n")
        
        # Create symbolic link to image
        src_img_path = os.path.join(image_dir, f"{image_id}.jpg")
        dst_img_path = os.path.join(output_dir, 'images', f"{image_id}.jpg")
        if os.path.exists(src_img_path):
            if not os.path.exists(dst_img_path):
                os.symlink(src_img_path, dst_img_path)
    
    # Process background images
    if background_images and background_dir:
        for image_id in tqdm(background_images, desc="Creating dataset - background"):
            src_img_path = os.path.join(background_dir, f"{image_id}.jpg")
            if not os.path.exists(src_img_path):
                continue
                
            # Create empty label file
            label_path = os.path.join(output_dir, 'labels', f"{image_id}.txt")
            open(label_path, 'w').close()  # Create empty file
            
            # Create symbolic link to background image
            dst_img_path = os.path.join(output_dir, 'images', f"{image_id}.jpg")
            if not os.path.exists(dst_img_path):
                os.symlink(src_img_path, dst_img_path)

def create_yaml_file(train_dir, val_dir, yaml_path, num_classes=1):
    """Create YAML configuration file for YOLOv5"""
    train_dir = os.path.abspath(train_dir)
    val_dir = os.path.abspath(val_dir)
    
    yaml_content = {
        'train': os.path.join(train_dir, 'images'),
        'val': os.path.join(val_dir, 'images'),
        'nc': num_classes,
        'names': [target_class]
    }
    
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f)

def get_train_val_split(filtered_annotations, box_count, fixed_val_size=500, random_state=42):
    """
    Get train/val split ensuring:
    1. Fixed number of validation boxes
    2. All boxes for same image kept together
    3. Consistent validation set across runs using random_state
    
    Args:
        filtered_annotations: DataFrame with all available annotations
        box_count: Target number of training boxes
        fixed_val_size: Target number of validation boxes
        random_state: Random seed for reproducibility
    """
    # First create validation set
    # Group by ImageID to keep all boxes for same image together
    grouped = filtered_annotations.groupby('ImageID')
    image_box_counts = grouped.size().sort_values(ascending=False)
    
    # Set random seed for validation selection
    np.random.seed(random_state)
    
    # Initialize validation set
    val_boxes = 0
    val_images = []
    remaining_images = list(image_box_counts.index)
    
    # Keep adding images until we reach or exceed fixed_val_size
    while val_boxes < fixed_val_size and remaining_images:
        # Randomly select an image
        img_idx = np.random.randint(len(remaining_images))
        img_id = remaining_images.pop(img_idx)
        
        # Add its boxes to validation count
        val_boxes += image_box_counts[img_id]
        val_images.append(img_id)
    
    print(f"Validation set: {val_boxes} boxes from {len(val_images)} images")
    
    # Remove validation images from available pool
    train_pool = image_box_counts.drop(val_images)
    
    # Now select training images up to target box count
    cumulative_boxes = train_pool.cumsum()
    selected_train_images = cumulative_boxes[cumulative_boxes <= box_count].index
    
    # If we need more boxes and have images available, add one more
    if len(selected_train_images) < len(train_pool) and sum(train_pool[selected_train_images]) < box_count:
        next_image = train_pool.index[len(selected_train_images)]
        selected_train_images = selected_train_images.append(pd.Index([next_image]))
    
    # Get final annotations
    train_annotations = filtered_annotations[filtered_annotations['ImageID'].isin(selected_train_images)]
    val_annotations = filtered_annotations[filtered_annotations['ImageID'].isin(val_images)]
    
    print(f"Training set: {len(train_annotations)} boxes from {len(selected_train_images)} images")
    print(f"Final validation set: {len(val_annotations)} boxes from {len(val_images)} images")
    
    return train_annotations, val_annotations

def train_yolo_model(box_count, image_dir, filtered_annotations, project_dir, fixed_val_size=500, max_epochs=500):
    """Main training function with fixed validation set size"""
    print(f"Training with target of {box_count} training boxes and {fixed_val_size} validation boxes")
    
    # Create directories
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(project_dir, f'{target_class}_run_{box_count}_boxes_{timestamp}')
    dataset_dir = os.path.join(run_dir, 'dataset')
    model_dir = os.path.join(run_dir, 'models')
    
    os.makedirs(run_dir, exist_ok=True)
    os.makedirs(dataset_dir, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)
    
    # Get train/val split with fixed validation size
    train_annotations, val_annotations = get_train_val_split(
        filtered_annotations, 
        box_count,
        fixed_val_size=fixed_val_size
    )
    
    # Get background images for training set
    train_image_count = len(train_annotations['ImageID'].unique())
    background_dir = r'D:\Data 255\image_data\random'
    
    if os.path.exists(background_dir):
        train_backgrounds = get_background_images(train_image_count, train_annotations, background_dir)
    else:
        train_backgrounds = []
        print("Background directory not found - proceeding without background images")
    
    # Create dataset structure
    train_dir = os.path.join(dataset_dir, 'train')
    val_dir = os.path.join(dataset_dir, 'val')
    
    # Create datasets
    create_yolo_dataset(train_annotations, image_dir, train_dir, 
                       background_images=train_backgrounds, 
                       background_dir=background_dir)
    create_yolo_dataset(val_annotations, image_dir, val_dir)
    
    # Create YAML configuration
    yaml_path = os.path.join(run_dir, 'dataset.yaml')
    create_yaml_file(train_dir, val_dir, yaml_path)
    
    # Initialize and train YOLOv8 model
    model = YOLO('yolov8s.pt')
    
    try:
        # Train the model
        results = model.train(
            data=yaml_path,
            epochs=max_epochs,
            imgsz=640,
            batch=16,
            workers=6,
            project=run_dir,
            name=f'yolo_boxes_{box_count}',
            patience=60,
            device=0,
            save=True,
            save_period=10,
            plots=True,
            verbose=True
        )
        
        # Save models
        yolo_output_dir = os.path.join(run_dir, f'yolo_boxes_{box_count}')
        
        # Save best and final models
        if os.path.exists(os.path.join(yolo_output_dir, 'weights', 'best.pt')):
            best_model_path = os.path.join(model_dir, f'model_best_boxes_{box_count}.pt')
            shutil.copy2(
                os.path.join(yolo_output_dir, 'weights', 'best.pt'),
                best_model_path
            )
        
        if os.path.exists(os.path.join(yolo_output_dir, 'weights', 'last.pt')):
            final_model_path = os.path.join(model_dir, f'model_final_boxes_{box_count}.pt')
            shutil.copy2(
                os.path.join(yolo_output_dir, 'weights', 'last.pt'),
                final_model_path
            )
        
        # Collect metrics
        metrics = {
            'BoundingBoxCount': box_count,
            'ValidationBoxCount': len(val_annotations),
            'TrainImageCount': len(train_annotations['ImageID'].unique()),
            'ValImageCount': len(val_annotations['ImageID'].unique()),
            'BatchSize': 16,
            'ModelDirectory': model_dir,
            'FinalEpoch': max_epochs
        }
        
        if hasattr(results, 'results_dict'):
            metrics_dict = results.results_dict
            metrics.update({
                'Precision': float(metrics_dict.get('metrics/precision', 0.0)),
                'Recall': float(metrics_dict.get('metrics/recall', 0.0)),
                'mAP50': float(metrics_dict.get('metrics/mAP50', 0.0)),
                'mAP50-95': float(metrics_dict.get('metrics/mAP50-95', 0.0))
            })
        
        return metrics
        
    except Exception as e:
        print(f"Error during training: {str(e)}")
        return {
            'BoundingBoxCount': box_count,
            'ValidationBoxCount': len(val_annotations),
            'TrainImageCount': len(train_annotations['ImageID'].unique()),
            'ValImageCount': len(val_annotations['ImageID'].unique()),
            'BatchSize': 16,
            'ModelDirectory': model_dir,
            'Precision': 0.0,
            'Recall': 0.0,
            'mAP50': 0.0,
            'mAP50-95': 0.0
        }

if __name__ == "__main__":
    # Set random seeds
    set_all_seeds()
    
    # Load and prepare data
    train_annotations = pd.read_csv('oidv6-train-annotations-bbox.csv')
    class_descriptions = pd.read_csv('oidv7-class-descriptions-boxable.csv')
    
    target_class = "Hat"
    BACKGROUND_PERCENTAGE = 20
    
    # Get target class label
    target_class_label = class_descriptions[class_descriptions['DisplayName'] == target_class]['LabelName'].values[0]
    
    # Set paths
    image_dir = rf'D:\Data 255\image_data\{target_class}'
    project_dir = r'D:\Data 255\yolo_training'
    
    # Filter annotations for target class
    filtered_annotations = train_annotations[train_annotations['LabelName'] == target_class_label]
    
    # Verify and filter based on available images
    filtered_annotations, available_images = verify_and_filter_data(filtered_annotations, image_dir)
    
    # Create project directory
    os.makedirs(project_dir, exist_ok=True)
    
    # Define box counts and train
    #bounding_box_counts = [100,300,500,750,1000,2000,3000,5000,10000,15000]  # Modify as needed
    bounding_box_counts = [16000]  # Modify as needed
    results = []
    
    for box_count in bounding_box_counts:
        try:
            metrics = train_yolo_model(box_count, image_dir, filtered_annotations, project_dir)
            results.append(metrics)
            
            # Save results
            results_path = os.path.join(project_dir, 'training_metrics.csv')
            save_metrics(results, results_path)
            
        except Exception as e:
            print(f"Error during training with {box_count} boxes:")
            print(str(e))
            continue
    
    print("All training complete!")

Original annotations: 13245
Available images: 8460
Filtered annotations: 13245
Unique images after filtering: 8460
Training with target of 16000 training boxes and 500 validation boxes
Validation set: 501 boxes from 339 images
Training set: 12744 boxes from 8121 images
Final validation set: 501 boxes from 339 images
Selected 1624 background images


Creating dataset - annotated: 100%|██████████| 339/339 [00:00<00:00, 430.20it/s]


New https://pypi.org/project/ultralytics/8.3.39 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.23  Python-3.10.0 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=D:\Data 255\yolo_training\Hat_run_16000_boxes_20241201_161705\dataset.yaml, epochs=500, time=None, patience=60, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=0, workers=6, project=D:\Data 255\yolo_training\Hat_run_16000_boxes_20241201_161705, name=yolo_boxes_16000, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fals

train: Scanning D:\Data 255\yolo_training\Hat_run_16000_boxes_20241201_161705\dataset\train\labels... 9745 images, 1624 backgrounds, 0 corrupt: 100%|██████████| 9745/9745 [02:58<00:00, 54.57it/s]


train: New cache created: D:\Data 255\yolo_training\Hat_run_16000_boxes_20241201_161705\dataset\train\labels.cache


...

    130/500      3.77G     0.7136     0.5092     0.9824          5        640: 100%|██████████| 610/610 [02:07<00:00,  4.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  3.48it/s]

                   all        339        501      0.889      0.888       0.92      0.763
EarlyStopping: Training stopped early as no improvement observed in last 60 epochs. Best results observed at epoch 70, best model saved as best.pt.
To update EarlyStopping(patience=60) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



130 epochs completed in 4.463 hours.
Optimizer stripped from D:\Data 255\yolo_training\Hat_run_16000_boxes_20241201_161705\yolo_boxes_16000\weights\last.pt, 22.5MB
Optimizer stripped from D:\Data 255\yolo_training\Hat_run_16000_boxes_20241201_161705\yolo_boxes_16000\weights\best.pt, 22.5MB

Validating D:\Data 255\yolo_training\Hat_run_16000_boxes_20241201_161705\yolo_boxes_16000\weights\best.pt...
Ultralytics 8.3.23  Python-3.10.0 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  2.82it/s]


                   all        339        501      0.883      0.877      0.925      0.767
Speed: 0.9ms preprocess, 2.2ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to D:\Data 255\yolo_training\Hat_run_16000_boxes_20241201_161705\yolo_boxes_16000
Results saved to D:\Data 255\yolo_training\training_metrics.csv
All training complete!
